In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1: Andy Ho (Team Lead)
- Name 2: Jaime Villanueva
- Name 3: Samira Zarandioon
- Name 4: Anjli Solsi

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
11,314
- What does each instance represent? 
Newsgroup Documents
- How many classes are in the dataset and what does each class represent?
20| Categories of News
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
We do not expect the classifer trained on this data would generalize since the documents are outdated.  The news has evolved and changed since then.
- Is the data represented as a sparse or dense matrix?
Sparse Matrix

___
Enter your answer here:

- 11,314 instances in the dataset
- Each instance represent newsgroup documents
- There are 20 classes, each representing the categories of news.
- We do not expect the classifer trained on this data would generalize since the documents are outdated.  The news has evolved and changed since then.
- Sparse Matrix





___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [37]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean (a,c), 'bc:',euclidean (b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine (b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard (a>0, b>0), 'ac:', jaccard (a>0, c>0), 'bc:', jaccard(b>0, c>0))

print('\n\nThe most appropriate distance is...')
print('Cosine is the most appropriate since it has the smallest value where bc = 0.42')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398236 bc: 0.9177794226661624
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156


The most appropriate distance is...
Cosine is the most appropriate since it has the smallest value where bc = 0.42


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [17]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
# K = 5 
for k in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
    clf = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    acc = clf.score (X_test, y_test)
    print('Accuracy of classifier with %d neighbors is: %.2f'%(k,acc))


acc = 0.2 # overwrite this with the actual accuracy

#=====================================

#print('Accuracy of classifier with %d neighbors is: %.2f'%(K,acc))


Accuracy of classifier with 1 neighbors is: 0.59
Accuracy of classifier with 2 neighbors is: 0.54
Accuracy of classifier with 3 neighbors is: 0.51
Accuracy of classifier with 4 neighbors is: 0.50
Accuracy of classifier with 5 neighbors is: 0.49
Accuracy of classifier with 6 neighbors is: 0.48
Accuracy of classifier with 7 neighbors is: 0.47
Accuracy of classifier with 8 neighbors is: 0.46
Accuracy of classifier with 9 neighbors is: 0.45
Accuracy of classifier with 10 neighbors is: 0.44
Accuracy of classifier with 11 neighbors is: 0.44
Accuracy of classifier with 12 neighbors is: 0.43
Accuracy of classifier with 13 neighbors is: 0.43
Accuracy of classifier with 14 neighbors is: 0.42
Accuracy of classifier with 15 neighbors is: 0.42
Accuracy of classifier with 16 neighbors is: 0.41
Accuracy of classifier with 17 neighbors is: 0.41
Accuracy of classifier with 18 neighbors is: 0.41
Accuracy of classifier with 19 neighbors is: 0.40
Accuracy of classifier with 20 neighbors is: 0.40


The best accuracy is 0.59 with 1 neighbor.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

It does not make sense.  A sparse matrix contains majority 0, the data would not branch and therefore would not have any sort of a KDTree.  A lot of the blocks would have 0s in them so it would not make sense to use KDtree in that regard.
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [26]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)

vector = ['l1', 'l2', 'cosine']

for c in vector : 
    clf = NearestCentroid(metric = c)
    clf.fit(X_train, y_train)
    acc = clf.score (X_test, y_test)
    #print('Accuracy of classifier with %d neighbors is: %.2f'% c,acc)
    print(c, acc)
    
# fill in your code here


print('The best distance metric is l1 with distance of:', 0.3077 )

/anaconda3/lib/python3.7/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


l1 0.30776029697719637
l2 0.39561605091037655
cosine 0.47516351423015735
The best distance metric is: l1 with distance of: 0.3077


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Total number of conditionals: 2,602,140 <p>
Priors need to be parameterized: 20 (1 for each class)

In [27]:
# Use this space for any calculations you might want to do
130107 * 20


2602140

___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

- Gaussian would be better for a instance containing mostly continous attributes.  For continous attributes, Bernouli probably would not be appropriate since Bernouli deals with Binary but Gaussian would work since it does well with large continous data - Central Limit Theorem could apply for a normal distribution.  Since this dataset is Sparse, having a lot of 0s, Guassian would not be practical.  
- Yes, if the data is sparse, it does change our answer. Bernoulli and Multinomial NB would probably be better. MultinomialNB works well with classification with descrete features for example word counts for text classification. 
___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [35]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

for j in [0, .005,  .01, .05, .1, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5] : 
    clf_mnb = MultinomialNB(alpha = j)
    clf_mnb.fit (X_train, y_train)
    acc = clf_mnb.score(X_test, y_test)
    print ("Multinomial NB alpha = ", j, "Accuracy is =", acc)

for k in [0, .005,  .01, .05, .1, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5] :   
    for m in [0, .005,  .01, .05, .1, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5] : 
        clf_bnb = BernoulliNB(alpha=k, binarize=m)
        clf_bnb.fit (X_train, y_train)
        acc = clf_bnb.score(X_test, y_test)
        print ("Bernoulli NB alpha = ", k, "Accuracy is =", acc)


# fill in your code here

            
print('These classifiers are so fast because...', p)
print('The alpha values control...', p)


/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Multinomial NB alpha =  0 Accuracy is = 0.8562842496022627
Multinomial NB alpha =  0.005 Accuracy is = 0.8829768428495669
Multinomial NB alpha =  0.01 Accuracy is = 0.8806788050203288
Multinomial NB alpha =  0.05 Accuracy is = 0.8531023510694714
Multinomial NB alpha =  0.1 Accuracy is = 0.8363089977019622
Multinomial NB alpha =  0.5 Accuracy is = 0.7613576100406576
Multinomial NB alpha =  1 Accuracy is = 0.7097401449531554
Multinomial NB alpha =  1.5 Accuracy is = 0.66572388191621
Multinomial NB alpha =  2 Accuracy is = 0.632313947321902
Multinomial NB alpha =  2.5 Accuracy is = 0.5976666077426198
Multinomial NB alpha =  3 Accuracy is = 0.5676153438218137
Multinomial NB alpha =  3.5 Accuracy is = 0.5395085734488244
Multinomial NB alpha =  4 Accuracy is = 0.5131695244829415
Multinomial NB alpha =  4.5 Accuracy is = 0.49231041187908786
Multinomial NB alpha =  5 Accuracy is = 0.4748099699487361
Bernoulli NB alpha =  0 Accuracy is = 0.848152731129574
Bernoulli NB alpha =  0 Accuracy is = 0

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.